# Assignment 3

In this assigment, we will work with the *Forest Fire* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/162/forest+fires). Extract the data files into the subdirectory: `../data/fires/` (relative to `./src/`).

## Objective

+ The model objective is to predict the area affected by forest fires given the features set. 
+ The objective of this exercise is to assess your ability to construct and evaluate model pipelines.
+ Please note: the instructions are not meant to be 100% prescriptive, but instead they are a set of minimum requirements. If you find predictive performance gains by applying additional steps, by all means show them. 

## Variable Description

From the description file contained in the archive (`forestfires.names`), we obtain the following variable descriptions:

1. X - x-axis spatial coordinate within the Montesinho park map: 1 to 9
2. Y - y-axis spatial coordinate within the Montesinho park map: 2 to 9
3. month - month of the year: "jan" to "dec" 
4. day - day of the week: "mon" to "sun"
5. FFMC - FFMC index from the FWI system: 18.7 to 96.20
6. DMC - DMC index from the FWI system: 1.1 to 291.3 
7. DC - DC index from the FWI system: 7.9 to 860.6 
8. ISI - ISI index from the FWI system: 0.0 to 56.10
9. temp - temperature in Celsius degrees: 2.2 to 33.30
10. RH - relative humidity in %: 15.0 to 100
11. wind - wind speed in km/h: 0.40 to 9.40 
12. rain - outside rain in mm/m2 : 0.0 to 6.4 
13. area - the burned area of the forest (in ha): 0.00 to 1090.84 









### Specific Tasks

+ Construct four model pipelines, out of combinations of the following components:

    + Preprocessors:

        - A simple processor that only scales numeric variables and recodes categorical variables.
        - A transformation preprocessor that scales numeric variables and applies a non-linear transformation.
    
    + Regressor:

        - A baseline regressor, which could be a [K-nearest neighbours model](https://open.spotify.com/track/4R3AU2pjv8ge2siX1fVbZs?si=b2712f32da0e4358) or a simple [linear regression model](https://scikit-learn.org/stable/modules/linear_model.html)
        - An advanced regressor of your choice (e.g., Random Forest, Neural Network, etc.)

+ Evaluate tune and evaluate each of the four model pipelines. 

    - Select a [performance metric](https://scikit-learn.org/stable/modules/linear_model.html) out of the following options: explained variance, max error, root mean squared error (RMSE), mean absolute error (MAE), r-squared.
    - *TIPS*: 
    
        * Out of the suggested metrics above, [some are correlation metrics, but this is a prediction problem](https://www.tmwr.org/performance#performance). Choose wisely (and don't choose the incorrect options.) 

+ Select the best-performing model and explain its predictions.

    - Provide local explanations.
    - Obtain global explanations and recommend a variable selection strategy.

+ Export your model as a pickle file.


You can work on the Jupyter notebook, as this experiment is fairly short (no need to use sacred). 

# Load the data

Assuming that the files forest data

In [78]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
from sklearn.preprocessing import RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import PowerTransformer
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.linear_model import ElasticNet

import warnings

warnings.filterwarnings('ignore')

In [79]:
columns = [
    'coord_x', 'coord_y', 'month', 'day', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain', 'area' 
]
fires_dt = (pd.read_csv('../../05_src/data/fires/forestfires.csv', header = None, names = columns))

fires_dt['month'].value_counts()   

month
aug      184
sep      172
mar       54
jul       32
feb       20
jun       17
oct       15
apr        9
dec        9
jan        2
may        2
month      1
nov        1
Name: count, dtype: int64

# Get X and Y

Create the features data frame and target data.

In [80]:
X = fires_dt.drop(columns='area')
Y = fires_dt['area']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)
scoring = ['neg_mean_absolute_error', 'neg_mean_squared_error', 'r2', 'explained_variance']

# Preprocessing

Create two [Column Transformers](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html), called preproc1 and preproc2, with the following guidelines:

- Numerical variables

    * (Preproc 1 and 2) Scaling: use a scaling method of your choice (Standard, Robust, Min-Max). 
    * Preproc 2 only: 
        
        + Choose a transformation for any of your input variables (or several of them). Evaluate if this transformation is convenient.
        + The choice of scaler is up to you.

- Categorical variables: 
    
    * (Preproc 1 and 2) Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) where appropriate.


+ The only difference between preproc1 and preproc2 is the non-linear transformation of the numerical variables.
    


In [81]:
num_features = ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain']
num_trans_features = ['ffmc', 'dmc', 'dc']
cat_features = ['month', 'day']



### Preproc 1

Create preproc1 below.

+ Numeric: scaled variables, no other transforms.
+ Categorical: one-hot encoding.

In [82]:
# preproc1
preproc1 = ColumnTransformer([
    ('robust_scaler',RobustScaler(),num_features),
    ('onehot', OneHotEncoder(handle_unknown='ignore'), cat_features)
])

preproc1

ColumnTransformer(transformers=[('robust_scaler', RobustScaler(),
                                 ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc',
                                  'isi', 'temp', 'rh', 'wind', 'rain']),
                                ('onehot',
                                 OneHotEncoder(handle_unknown='ignore'),
                                 ['month', 'day'])])

### Preproc 2

Create preproc1 below.

+ Numeric: scaled variables, non-linear transformation to one or more variables.
+ Categorical: one-hot encoding.

In [83]:
# preproc2
preproc2 = ColumnTransformer([
    ('standard_scaler', StandardScaler(), num_trans_features),
    ('transform', PowerTransformer(method='yeo-johnson'), num_trans_features),
    ('onehot', OneHotEncoder(handle_unknown='ignore'), cat_features)
])

preproc2

ColumnTransformer(transformers=[('standard_scaler', StandardScaler(),
                                 ['ffmc', 'dmc', 'dc']),
                                ('transform', PowerTransformer(),
                                 ['ffmc', 'dmc', 'dc']),
                                ('onehot',
                                 OneHotEncoder(handle_unknown='ignore'),
                                 ['month', 'day'])])

## Model Pipeline


Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `regressor` and assign a regression model to it. 

## Regressor

Question is it GridSearchCV()?

+ Use a regression model to perform a prediction. 

    - Choose a baseline regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Choose a more advance regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Both model choices are up to you, feel free to experiment.

In [84]:
# Pipeline A = preproc1 + baseline
pipe_A = Pipeline([
    ('preprocess1', preproc1),
    ('regressor1', ElasticNet())
])

param_grid1 = {
    'regressor1__alpha': [0.01, 0.1, 1, 10],
    'regressor1__l1_ratio': [0.1, 0.5, 1],
    'regressor1__max_iter': [1000, 5000, 10000]
}

grid_cv1 = GridSearchCV(
    estimator=pipe_A, 
    param_grid=param_grid1, 
    scoring = scoring, 
    cv = 5,
    refit = 'neg_mean_squared_error')

grid_cv1.fit(X_train, Y_train)



GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocess1',
                                        ColumnTransformer(transformers=[('robust_scaler',
                                                                         RobustScaler(),
                                                                         ['coord_x',
                                                                          'coord_y',
                                                                          'ffmc',
                                                                          'dmc',
                                                                          'dc',
                                                                          'isi',
                                                                          'temp',
                                                                          'rh',
                                                                          'wind',
                                                                          'rain']),
                                                                        ('onehot',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['month',
                                                                          'day'])])),
                                       ('regressor1', ElasticNet())]),
             param_grid={'regressor1__alpha': [0.01, 0.1, 1, 10],
                         'regressor1__l1_ratio': [0.1, 0.5, 1],
                         'regressor1__max_iter': [1000, 5000, 10000]},
             refit='neg_mean_squared_error',
             scoring=['neg_mean_absolute_error', 'neg_mean_squared_error', 'r2',
                      'explained_variance'])

In [85]:
grid_res_A = grid_cv1.cv_results_
grid_res_A = pd.DataFrame(grid_res_A)
grid_res_A.columns

grid_res_A[['mean_fit_time','mean_score_time','param_regressor1__alpha','param_regressor1__l1_ratio',
            'param_regressor1__max_iter','mean_test_neg_mean_absolute_error',
            'mean_test_neg_mean_squared_error','mean_test_r2', 'mean_test_explained_variance',
            'rank_test_neg_mean_squared_error']].sort_values('rank_test_neg_mean_squared_error')

,mean_fit_time,mean_score_time,param_regressor1__alpha,param_regressor1__l1_ratio,param_regressor1__max_iter,mean_test_neg_mean_absolute_error,mean_test_neg_mean_squared_error,mean_test_r2,mean_test_explained_variance,rank_test_neg_mean_squared_error
29,0.006743,0.003764,10,0.1,10000,-23.137419,-5664.945002,-0.042577,-2.491230e-03,1
28,0.006671,0.003860,10,0.1,5000,-23.137419,-5664.945002,-0.042577,-2.491230e-03,1
27,0.006551,0.003836,10,0.1,1000,-23.137419,-5664.945002,-0.042577,-2.491230e-03,1
35,0.006779,0.003925,10,1,10000,-23.095694,-5667.588020,-0.042239,4.440892e-17,4
33,0.006955,0.004097,10,1,1000,-23.095694,-5667.588020,-0.042239,4.440892e-17,4
34,0.006783,0.003781,10,1,5000,-23.095694,-5667.588020,-0.042239,4.440892e-17,4
32,0.006940,0.003903,10,0.5,10000,-23.152445,-5672.519492,-0.045401,-3.554030e-03,7
31,0.006979,0.003910,10,0.5,5000,-23.152445,-5672.519492,-0.045401,-3.554030e-03,7
30,0.006498,0.003919,10,0.5,1000,-23.152445,-5672.519492,-0.045401,-3.554030e-03,7
20,0.006649,0.003734,1,0.1,10000,-23.539003,-5680.015232,-0.071270,-3.473737e-02,10


In [86]:
best_params_A = grid_cv1.best_params_

pipe_A.set_params(**best_params_A)

res_A = cross_validate(pipe_A, X_train, Y_train, cv = 5, scoring = scoring)
res_A = pd.DataFrame(res_A).assign(experiment = 1)
res_A

,fit_time,score_time,test_neg_mean_absolute_error,test_neg_mean_squared_error,test_r2,test_explained_variance,experiment
0,0.009994,0.006066,-21.147104,-1385.184076,-0.055464,-0.027719,1
1,0.010010,0.005911,-28.259825,-16158.985273,-0.003426,0.001300,1
2,0.009272,0.005774,-27.108478,-8225.460293,-0.005884,0.001586,1
3,0.007041,0.004506,-22.019117,-1989.260655,0.000615,0.002348,1
4,0.007599,0.003747,-17.152571,-565.834714,-0.148727,0.010028,1


In [87]:
res_A.mean()

fit_time                           0.008783
score_time                         0.005201
test_neg_mean_absolute_error     -23.137419
test_neg_mean_squared_error    -5664.945002
test_r2                           -0.042577
test_explained_variance           -0.002491
experiment                         1.000000
dtype: float64

In [88]:
# Pipeline B = preproc2 + baseline
pipe_B = Pipeline([
    ('preprocess2', preproc2), 
    ('regressor2', ElasticNet())
])

param_grid2 = {
    'regressor2__alpha': [0.01, 0.1, 1, 10],
    'regressor2__l1_ratio': [0.1, 0.5, 1],
    'regressor2__max_iter': [1000, 5000, 10000]
}

grid_cv2 = GridSearchCV(
    estimator=pipe_B, 
    param_grid=param_grid2, 
    scoring = scoring, 
    cv = 5,
    refit = 'neg_mean_squared_error')

grid_cv2.fit(X_train, Y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocess2',
                                        ColumnTransformer(transformers=[('standard_scaler',
                                                                         StandardScaler(),
                                                                         ['ffmc',
                                                                          'dmc',
                                                                          'dc']),
                                                                        ('transform',
                                                                         PowerTransformer(),
                                                                         ['ffmc',
                                                                          'dmc',
                                                                          'dc']),
                                                                        ('onehot',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['month',
                                                                          'day'])])),
                                       ('regressor2', ElasticNet())]),
             param_grid={'regressor2__alpha': [0.01, 0.1, 1, 10],
                         'regressor2__l1_ratio': [0.1, 0.5, 1],
                         'regressor2__max_iter': [1000, 5000, 10000]},
             refit='neg_mean_squared_error',
             scoring=['neg_mean_absolute_error', 'neg_mean_squared_error', 'r2',
                      'explained_variance'])

In [89]:
grid_res_B = grid_cv2.cv_results_
grid_res_B = pd.DataFrame(grid_res_B)
grid_res_B.columns

grid_res_B[['mean_fit_time','mean_score_time','param_regressor2__alpha','param_regressor2__l1_ratio',
            'param_regressor2__max_iter','mean_test_neg_mean_absolute_error',
            'mean_test_neg_mean_squared_error','mean_test_r2', 'mean_test_explained_variance',
            'rank_test_neg_mean_squared_error']].sort_values('rank_test_neg_mean_squared_error')

,mean_fit_time,mean_score_time,param_regressor2__alpha,param_regressor2__l1_ratio,param_regressor2__max_iter,mean_test_neg_mean_absolute_error,mean_test_neg_mean_squared_error,mean_test_r2,mean_test_explained_variance,rank_test_neg_mean_squared_error
29,0.010382,0.004898,10,0.1,10000,-23.122576,-5663.697500,-0.039251,1.447094e-03,1
28,0.009726,0.005696,10,0.1,5000,-23.122576,-5663.697500,-0.039251,1.447094e-03,1
27,0.010188,0.005435,10,0.1,1000,-23.122576,-5663.697500,-0.039251,1.447094e-03,1
32,0.009844,0.004337,10,0.5,10000,-23.108886,-5666.285233,-0.040807,9.161203e-04,4
31,0.009596,0.004542,10,0.5,5000,-23.108886,-5666.285233,-0.040807,9.161203e-04,4
30,0.009896,0.004380,10,0.5,1000,-23.108886,-5666.285233,-0.040807,9.161203e-04,4
35,0.009698,0.004578,10,1,10000,-23.095694,-5667.588020,-0.042239,4.440892e-17,7
33,0.009682,0.004533,10,1,1000,-23.095694,-5667.588020,-0.042239,4.440892e-17,7
34,0.009805,0.004380,10,1,5000,-23.095694,-5667.588020,-0.042239,4.440892e-17,7
20,0.010061,0.004947,1,0.1,10000,-23.322608,-5671.162123,-0.038230,-9.942957e-04,10


In [90]:
best_params_B = grid_cv2.best_params_
pipe_B.set_params(**best_params_B)

res_B = cross_validate(pipe_B, X_train, Y_train, cv = 5, scoring = scoring)
res_B = pd.DataFrame(res_B).assign(experiment = 1)
res_B

,fit_time,score_time,test_neg_mean_absolute_error,test_neg_mean_squared_error,test_r2,test_explained_variance,experiment
0,0.014824,0.005211,-20.784865,-1344.810018,-0.024700,0.001839,1
1,0.010525,0.004547,-28.293604,-16178.794430,-0.004656,0.000132,1
2,0.010333,0.004793,-27.120767,-8232.290100,-0.006720,0.000759,1
3,0.010764,0.004452,-22.123667,-1991.326860,-0.000423,0.001546,1
4,0.010780,0.004455,-17.289977,-571.266093,-0.159754,0.002960,1


In [91]:
res_B.mean()

fit_time                           0.011445
score_time                         0.004691
test_neg_mean_absolute_error     -23.122576
test_neg_mean_squared_error    -5663.697500
test_r2                           -0.039251
test_explained_variance            0.001447
experiment                         1.000000
dtype: float64

In [92]:
# Pipeline C = preproc1 + advanced model
pipe_C = Pipeline([
    ('preprocess3', preproc1),
    ('regressor3', RandomForestRegressor())
])

param_grid3 = {
    'regressor3__n_estimators': [100, 300, 500, 700, 1000],
    'regressor3__max_depth': [None, 10, 20, 30, 40, 50],
    'regressor3__max_features': ['auto', 'sqrt', 'log2']
}

grid_cv3 = GridSearchCV(
    estimator=pipe_C, 
    param_grid=param_grid3, 
    scoring = scoring, 
    cv = 5,
    refit = 'neg_mean_squared_error')

grid_cv3.fit(X_train, Y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocess3',
                                        ColumnTransformer(transformers=[('robust_scaler',
                                                                         RobustScaler(),
                                                                         ['coord_x',
                                                                          'coord_y',
                                                                          'ffmc',
                                                                          'dmc',
                                                                          'dc',
                                                                          'isi',
                                                                          'temp',
                                                                          'rh',
                                                                          'wind',
                                                                          'rain']),
                                                                        ('onehot',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['month',
                                                                          'day'])])),
                                       ('regressor3',
                                        RandomForestRegressor())]),
             param_grid={'regressor3__max_depth': [None, 10, 20, 30, 40, 50],
                         'regressor3__max_features': ['auto', 'sqrt', 'log2'],
                         'regressor3__n_estimators': [100, 300, 500, 700,
                                                      1000]},
             refit='neg_mean_squared_error',
             scoring=['neg_mean_absolute_error', 'neg_mean_squared_error', 'r2',
                      'explained_variance'])

In [93]:
grid_res_C = grid_cv3.cv_results_
grid_res_C = pd.DataFrame(grid_res_C)
grid_res_C.columns

grid_res_C[['mean_fit_time','mean_score_time','param_regressor3__max_depth', 
            'param_regressor3__max_features','param_regressor3__n_estimators',
            'mean_test_neg_mean_absolute_error','mean_test_neg_mean_squared_error','mean_test_r2',
            'mean_test_explained_variance',
            'rank_test_neg_mean_squared_error']].sort_values('rank_test_neg_mean_squared_error')

,mean_fit_time,mean_score_time,param_regressor3__max_depth,param_regressor3__max_features,param_regressor3__n_estimators,mean_test_neg_mean_absolute_error,mean_test_neg_mean_squared_error,mean_test_r2,mean_test_explained_variance,rank_test_neg_mean_squared_error
55,0.091294,0.007480,30,log2,100,-24.397536,-5780.016456,-0.239590,-0.175604,1
25,0.088455,0.007208,10,log2,100,-24.861335,-5793.499211,-0.257174,-0.179507,2
44,0.837827,0.035503,20,log2,1000,-24.926919,-5833.193824,-0.223955,-0.149054,3
28,0.560480,0.024750,10,log2,700,-25.043162,-5834.186943,-0.246802,-0.167578,4
26,0.245195,0.013019,10,log2,300,-24.671245,-5838.441088,-0.208651,-0.132638,5
...,...,...,...,...,...,...,...,...,...,...
60,0.005795,0.000000,40,auto,100,NaN,NaN,NaN,NaN,61
61,0.005720,0.000000,40,auto,300,NaN,NaN,NaN,NaN,61
62,0.005727,0.000000,40,auto,500,NaN,NaN,NaN,NaN,61
64,0.005754,0.000000,40,auto,1000,NaN,NaN,NaN,NaN,61


In [94]:
grid_cv3.best_params_

{'regressor3__max_depth': 30,
 'regressor3__max_features': 'log2',
 'regressor3__n_estimators': 100}

In [95]:
grid_cv3.best_estimator_

Pipeline(steps=[('preprocess3',
                 ColumnTransformer(transformers=[('robust_scaler',
                                                  RobustScaler(),
                                                  ['coord_x', 'coord_y', 'ffmc',
                                                   'dmc', 'dc', 'isi', 'temp',
                                                   'rh', 'wind', 'rain']),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['month', 'day'])])),
                ('regressor3',
                 RandomForestRegressor(max_depth=30, max_features='log2'))])

In [96]:
best_params_C = grid_cv3.best_params_

pipe_C.set_params(**best_params_C)

res_C = cross_validate(pipe_C, X_train, Y_train, cv = 5, scoring = scoring)
res_C = pd.DataFrame(res_C).assign(experiment = 1)
res_C

,fit_time,score_time,test_neg_mean_absolute_error,test_neg_mean_squared_error,test_r2,test_explained_variance,experiment
0,0.165146,0.009561,-28.011308,-2662.470045,-1.028713,-0.888789,1
1,0.096755,0.009397,-30.526182,-15470.699429,0.039314,0.042325,1
2,0.096395,0.007201,-29.159378,-8876.665612,-0.085520,-0.078364,1
3,0.095726,0.008068,-21.034656,-2319.379537,-0.165233,-0.162385,1
4,0.093988,0.007543,-14.127954,-452.201841,0.081964,0.237797,1


In [97]:
res_C.mean()

fit_time                           0.109602
score_time                         0.008354
test_neg_mean_absolute_error     -24.571895
test_neg_mean_squared_error    -5956.283293
test_r2                           -0.231637
test_explained_variance           -0.169883
experiment                         1.000000
dtype: float64

In [98]:
# Pipeline D = preproc2 + advanced model
pipe_D = Pipeline([
    ('preprocess4', preproc2), 
    ('regressor4', RandomForestRegressor())
])


param_grid4 = {
    'regressor4__n_estimators': [100, 300, 500, 700, 1000],
    'regressor4__max_depth': [None, 10, 20, 30, 40, 50],
    'regressor4__max_features': ['auto', 'sqrt', 'log2']
}

grid_cv4 = GridSearchCV(
    estimator=pipe_D, 
    param_grid=param_grid4, 
    scoring = scoring, 
    cv = 5,
    refit = 'neg_mean_squared_error')

grid_cv4.fit(X_train, Y_train)


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocess4',
                                        ColumnTransformer(transformers=[('standard_scaler',
                                                                         StandardScaler(),
                                                                         ['ffmc',
                                                                          'dmc',
                                                                          'dc']),
                                                                        ('transform',
                                                                         PowerTransformer(),
                                                                         ['ffmc',
                                                                          'dmc',
                                                                          'dc']),
                                                                        ('onehot',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['month',
                                                                          'day'])])),
                                       ('regressor4',
                                        RandomForestRegressor())]),
             param_grid={'regressor4__max_depth': [None, 10, 20, 30, 40, 50],
                         'regressor4__max_features': ['auto', 'sqrt', 'log2'],
                         'regressor4__n_estimators': [100, 300, 500, 700,
                                                      1000]},
             refit='neg_mean_squared_error',
             scoring=['neg_mean_absolute_error', 'neg_mean_squared_error', 'r2',
                      'explained_variance'])

In [99]:
grid_res_D = grid_cv4.cv_results_
grid_res_D = pd.DataFrame(grid_res_D)
grid_res_D.columns

grid_res_D[['mean_fit_time','mean_score_time','param_regressor4__max_depth',
            'param_regressor4__max_features','param_regressor4__n_estimators',
            'mean_test_neg_mean_absolute_error', 'mean_test_neg_mean_squared_error'
            ,'mean_test_r2', 'mean_test_explained_variance'
            ,'rank_test_neg_mean_squared_error']].sort_values('rank_test_neg_mean_squared_error')

,mean_fit_time,mean_score_time,param_regressor4__max_depth,param_regressor4__max_features,param_regressor4__n_estimators,mean_test_neg_mean_absolute_error,mean_test_neg_mean_squared_error,mean_test_r2,mean_test_explained_variance,rank_test_neg_mean_squared_error
5,0.096241,0.008076,None,sqrt,100,-25.033268,-6396.242606,-0.464386,-0.396369,1
86,0.267338,0.014636,50,log2,300,-24.776125,-6593.603046,-0.568623,-0.503745,2
29,0.802686,0.034656,10,log2,1000,-24.911098,-6596.568814,-0.536204,-0.472881,3
51,0.260210,0.013976,30,sqrt,300,-25.180946,-6605.477351,-0.466819,-0.408538,4
50,0.094484,0.007995,30,sqrt,100,-25.005692,-6607.082897,-0.477168,-0.415258,5
...,...,...,...,...,...,...,...,...,...,...
48,0.009193,0.000000,30,auto,700,NaN,NaN,NaN,NaN,61
49,0.009288,0.000000,30,auto,1000,NaN,NaN,NaN,NaN,61
17,0.009148,0.000000,10,auto,500,NaN,NaN,NaN,NaN,61
15,0.009150,0.000000,10,auto,100,NaN,NaN,NaN,NaN,61


In [100]:
best_params_D = grid_cv4.best_params_
pipe_D.set_params(**best_params_D)

res_D = cross_validate(pipe_D, X_train, Y_train, cv = 5, scoring = scoring)
res_D = pd.DataFrame(res_D).assign(experiment = 1)
res_D.mean()


fit_time                           0.110897
score_time                         0.009068
test_neg_mean_absolute_error     -26.052143
test_neg_mean_squared_error    -6940.266591
test_r2                           -0.839136
test_explained_variance           -0.762315
experiment                         1.000000
dtype: float64

# Tune Hyperparams

+ Perform GridSearch on each of the four pipelines. 
+ Tune at least one hyperparameter per pipeline.
+ Experiment with at least four value combinations per pipeline.

# Evaluate

+ Which model has the best performance?

In [101]:


pipes = {
    'pipe_A': pipe_A,
    'pipe_B': pipe_B,
    'pipe_C': pipe_C,
    'pipe_D': pipe_D
}

for pipe_name, pipe in pipes.items():
    
    y_pred = pipe.predict(X_test)
    mse = mean_squared_error(Y_test, y_pred)
    mae = mean_absolute_error(Y_test, y_pred)
    r2 = r2_score(Y_test, y_pred)
    print(f"{pipe_name}: Mean Squared Error on test set = {mse}")
    print(f"{pipe_name}: Mean Absolute Error on test set = {mae}")
    print(f"{pipe_name}: R2 on test set = {r2}")

    
    



NotFittedError: This ColumnTransformer instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

# Export

+ Save the best performing model to a pickle file.

# Explain

+ Use SHAP values to explain the following only for the best-performing model:

    - Select an observation in your test set and explain which are the most important features that explain that observation's specific prediction.

    - In general, across the complete training set, which features are the most and least important.

+ If you were to remove features from the model, which ones would you remove? Why? How would you test that these features are actually enhancing model performance?

*(Answer here.)*

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-3`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_3.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Cortez,Paulo and Morais,Anbal. (2008). Forest Fires. UCI Machine Learning Repository. https://doi.org/10.24432/C5D88D.